In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.default.active_users_history AS
SELECT
  year_month,
  active_users,
  LAG(active_users, 1) OVER (ORDER BY year_month) AS active_users_previous_month,
  CASE
    WHEN LAG(active_users, 1) OVER (ORDER BY year_month) IS NULL THEN 0
    ELSE active_users - LAG(active_users, 1) OVER (ORDER BY year_month)
  END AS growth,
  ROUND(
    ((active_users - LAG(active_users, 1) OVER (ORDER BY year_month)) * 100)
    / NULLIF(LAG(active_users, 1) OVER (ORDER BY year_month), 0),
    2
  ) AS perc_grow
FROM
  (
    SELECT
      date_format(event_time, 'yyyy-MM') AS year_month,
      COUNT(DISTINCT COALESCE(user_identity.email, request_params.user)) AS active_users
    FROM
      system.access.audit
    WHERE
      action_name IS NOT NULL
      AND COALESCE(user_identity.email, request_params.user) IS NOT NULL
      AND (
        user_agent LIKE '%Mozilla%'
        OR user_agent LIKE '%Chrome%'
        OR user_agent LIKE '%Safari%'
        OR user_agent LIKE '%AppleWebKit%'
        OR user_agent LIKE '%Edge%'
      )
      AND workspace_id = dataops_prd.libs.get_workspace_id()
    GROUP BY
      year_month
  ) t
ORDER BY
  year_month

In [0]:
df = spark.sql("""
SELECT
  year_month,
  active_users,
  LAG(active_users, 1) OVER (ORDER BY year_month) AS active_users_previous_month,
  CASE
    WHEN LAG(active_users, 1) OVER (ORDER BY year_month) IS NULL THEN 0
    ELSE active_users - LAG(active_users, 1) OVER (ORDER BY year_month)
  END AS growth,
  ROUND(
    ((active_users - LAG(active_users, 1) OVER (ORDER BY year_month)) * 100)
    / NULLIF(LAG(active_users, 1) OVER (ORDER BY year_month), 0),
    2
  ) AS perc_grow
FROM
  (
    SELECT
      date_format(event_time, 'yyyy-MM') AS year_month,
      COUNT(DISTINCT COALESCE(user_identity.email, request_params.user)) AS active_users
    FROM
      system.access.audit
    WHERE
      action_name IS NOT NULL
      AND COALESCE(user_identity.email, request_params.user) IS NOT NULL
      AND (
        user_agent LIKE '%Mozilla%'
        OR user_agent LIKE '%Chrome%'
        OR user_agent LIKE '%Safari%'
        OR user_agent LIKE '%AppleWebKit%'
        OR user_agent LIKE '%Edge%'
      )
      AND workspace_id = dataops_prd.libs.get_workspace_id()
    GROUP BY
      year_month
  ) t
ORDER BY
  year_month
               """)

In [0]:
df = df.filter("year_month = date_format(current_date(),'yyyy-MM')")

In [0]:
spark.sql("""
MERGE INTO workspace.default.active_users_history AS target
USING {df} AS source
ON target.year_month = source.year_month
WHEN MATCHED THEN
  UPDATE SET
    *
WHEN NOT MATCHED THEN
  INSERT *
""",df=df)

In [0]:
%sql
select * from workspace.default.active_users_history